In [0]:
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("sample_submission.csv")

In [0]:
train_30 =train.iloc[:4320]
train_30

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
0,0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,0.0,0.00,82.4,71.3,0.0,101.1,0.0,56.1,0.3,0.0,91.0,0.0,1002.5,1009.8,1.1,2.3,0.5,1008.7,12.3,1001.6,69.1,8.2,10.7,1010.1,0.00,256.4,0.0,77.2,62.6,0.0,12.5,11.5,11.0,20.0,20.0,10.5,10.0,10.5,10.0,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,NaN
1,1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,0.0,0.00,81.2,121.8,0.0,165.3,0.0,0.0,0.4,0.0,91.9,0.0,1002.5,1009.9,0.0,0.0,0.0,1008.7,12.1,1001.7,70.3,8.3,10.3,1010.1,0.00,215.4,0.0,77.3,63.5,0.0,12.5,11.0,11.0,20.0,20.0,10.0,9.5,10.0,9.5,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,NaN
2,2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,0.0,0.00,86.1,232.0,0.0,144.0,0.0,0.0,0.6,0.0,93.3,0.0,1002.4,1009.8,0.0,0.0,0.0,1008.6,12.1,1001.6,71.5,8.0,9.7,1010.0,0.00,235.2,0.0,77.3,63.9,0.0,12.0,11.0,11.0,19.5,19.5,10.0,9.0,10.0,9.0,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,NaN
3,3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,0.0,0.00,87.7,0.0,0.0,115.9,0.0,294.1,0.1,0.0,93.4,0.0,1002.4,1009.7,0.0,0.0,0.2,1008.7,12.1,1001.5,73.2,7.7,9.4,1010.1,0.00,214.0,0.0,77.5,64.5,0.0,12.0,11.0,11.0,19.5,19.5,9.5,9.0,9.5,9.0,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,NaN
4,4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,0.0,0.00,88.9,0.0,0.0,124.7,0.0,0.0,0.0,0.0,94.1,0.0,1002.4,1009.7,0.0,0.0,0.0,1008.7,12.0,1001.5,74.3,7.4,9.4,1010.1,0.00,174.9,0.0,78.0,65.0,0.0,12.0,10.5,10.5,19.0,19.5,9.5,8.5,9.5,8.5,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,4315,19.5,987.8,1.7,0.4,0.0,1007.8,988.0,21.3,1007.8,1007.5,0.0,13.21,92.0,227.5,0.0,304.0,0.0,10.7,0.4,0.0,84.1,0.0,1001.2,1008.2,0.4,328.4,0.6,1006.6,21.6,1000.1,82.3,18.8,19.4,1008.2,16.68,272.9,0.0,82.3,74.3,0.0,22.0,22.5,22.5,26.0,26.0,21.0,20.0,20.5,20.5,19.5,20.0,19.0,20.5,20.5,20.0,20.0,20.5,20.0,NaN
4316,4316,19.3,987.7,2.1,0.9,0.0,1007.8,988.1,21.3,1008.0,1007.4,0.0,13.21,92.0,259.4,0.0,309.1,0.0,0.7,0.3,0.0,85.7,0.0,1001.2,1008.2,0.2,346.7,1.1,1006.8,21.6,1000.1,85.8,18.6,18.5,1008.4,16.68,315.9,0.0,82.4,74.8,0.0,22.0,22.5,22.5,26.0,25.5,21.0,20.0,20.5,20.5,19.5,20.0,19.0,20.5,20.5,20.0,19.5,20.5,19.5,NaN
4317,4317,19.5,987.8,0.9,1.3,0.0,1007.8,988.3,21.2,1008.1,1007.5,0.0,13.21,91.9,256.1,0.0,318.6,0.0,54.7,0.9,0.0,85.9,0.0,1001.2,1008.3,0.9,342.1,0.5,1006.8,21.5,1000.2,84.1,18.4,19.1,1008.4,16.68,343.3,0.0,82.8,75.4,0.0,21.5,22.5,22.5,26.0,25.5,21.0,20.0,20.5,20.5,19.5,20.0,19.0,20.0,20.0,19.5,19.5,20.0,19.0,NaN
4318,4318,20.0,987.8,1.4,0.5,0.0,1007.8,988.3,21.1,1008.1,1007.5,0.0,13.21,92.1,203.9,0.0,327.7,0.0,116.1,0.4,0.0,83.7,0.0,1001.2,1008.1,0.7,345.5,0.5,1006.8,21.3,1000.0,85.4,18.2,19.0,1008.4,16.68,341.7,0.0,82.8,75.8,0.0,21.5,22.0,22.0,25.5,25.5,20.5,20.0,20.0,20.0,19.5,20.0,18.5,20.0,20.0,19.5,19.5,20.0,19.0,NaN


In [0]:

train_30 = pd.concat([train_30,train_30],axis=0)

In [0]:
train_3 = train.loc[4320:]
train = pd.concat([train_30,train_3],axis=0)

In [0]:
t = train["Y18"].isnull()
index_null = t[t==True].index
train.loc[index_null,'Y18'] = train.loc[index_null,'Y16']

In [0]:
test = test.loc[:, test.min() != test.max()]
train = train.loc[:, train.min() != train.max()]

In [0]:
x=train.loc[:,"X00":"X39"]
y=train.loc[:,"Y18"]

In [0]:
x["X40"] = x.loc[:,['X00','X07','X31','X32']].mean(axis=1)
x["X41"] = x.loc[:,['X18','X24']].mean(axis=1)

In [0]:
test["X40"] = test.loc[:,['X00','X07','X31','X32']].mean(axis=1)
test["X41"] = test.loc[:,['X18','X24']].mean(axis=1)

In [0]:
def standardization(df):
  mean = np.mean(df)
  std = np.std(df)
  norm = (df - mean) / (std - 1e-07)
  return norm, mean, std

In [0]:
X_train_norm, mean, std = standardization(x)
X_test = (test - mean) / (std - 1e-07)

In [0]:
# ID로 시간 변수 생성
minute = pd.Series((X_train_norm.index%144).astype(int))
hour= pd.Series((X_train_norm.index%144/6).astype(int))

# 삼각함수를 이용한 시간변수 생성
min_in_day = 24*6
hour_in_day = 24

minute_sin = np.sin(np.pi*minute/min_in_day) 
minute_cos = np.cos(np.pi*minute/min_in_day)

hour_sin  = np.sin(np.pi*hour/hour_in_day)
hour_cos  = np.cos(np.pi*hour/hour_in_day)

In [0]:
X_train_norm['minute_sin'] = minute_sin
X_train_norm['minute_cos'] = minute_cos

X_train_norm['hour_sin'] = hour_sin
X_train_norm['hour_cos'] = hour_cos
X_train_norm

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X15,X17,X18,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,minute_sin,minute_cos,hour_sin,hour_cos
0,-2.200858,0.021446,-0.302269,-0.901206,-0.294097,0.250176,0.140271,-2.125617,0.278398,0.359492,-0.308606,-1.009584,0.641257,-0.914050,-0.734332,-0.899913,-0.932557,0.889403,-0.280801,0.182418,0.242728,0.107266,-1.038180,-0.692139,0.296425,-2.280770,0.198271,0.167066,-2.071975,-1.731011,0.262630,-1.012858,0.492439,-0.281954,0.584950,0.253448,-0.239790,-2.051551,-0.476449,0.000000,1.000000,0.000000,1.000000
1,-2.285482,0.050911,0.160625,0.473953,-0.294097,0.250176,0.140271,-2.148344,0.306617,0.387481,-0.308606,-1.009584,0.587833,-0.400739,-0.072450,-1.387470,-0.846263,0.930206,-0.280801,0.182418,0.271742,-0.897265,-1.055778,-1.091776,0.296425,-2.327830,0.227499,0.222209,-2.054006,-1.806372,0.262630,-1.012858,0.076828,-0.281954,0.590018,0.302031,-0.239790,-2.092101,-0.971093,0.021815,0.999762,0.000000,1.000000
2,-2.264326,0.080376,-0.394848,0.897079,-0.294097,0.220989,0.169400,-2.148344,0.334836,0.443458,-0.308606,-1.009584,0.805978,0.719397,-0.292047,-1.387470,-0.673675,0.993677,-0.280801,0.153012,0.242728,-0.897265,-1.055778,-1.091776,0.267662,-2.327830,0.198271,0.277353,-2.107912,-1.919412,0.233818,-1.012858,0.277538,-0.281954,0.590018,0.323624,-0.239790,-2.132651,-0.872164,0.043619,0.999048,0.000000,1.000000
3,-2.264326,0.050911,-0.024533,-0.795425,-0.294097,0.220989,0.140271,-2.171071,0.306617,0.415469,-0.308606,-1.009584,0.877209,-1.638784,-0.581749,1.168513,-1.105144,0.998210,-0.280801,0.153012,0.213715,-0.897265,-1.055778,-0.931921,0.296425,-2.327830,0.169043,0.355474,-2.161819,-1.975933,0.262630,-1.012858,0.062636,-0.281954,0.600155,0.356013,-0.239790,-2.168132,-1.119486,0.065403,0.997859,0.000000,1.000000
4,-2.306638,0.050911,-0.672584,0.262390,-0.294097,0.220989,0.169400,-2.171071,0.334836,0.415469,-0.308606,-1.009584,0.930632,-1.638784,-0.491024,-1.387470,-1.191438,1.029946,-0.280801,0.153012,0.213715,-0.897265,-1.055778,-1.091776,0.296425,-2.351360,0.169043,0.406023,-2.215725,-1.975933,0.262630,-1.012858,-0.333715,-0.281954,0.625499,0.383004,-0.239790,-2.193476,-1.168950,0.087156,0.996195,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,-0.042946,-0.332135,-0.580005,-0.689643,-0.294097,-0.450307,-0.413183,0.033452,-0.398857,-0.312228,0.427879,1.177892,1.215556,1.747035,1.057492,1.006863,-0.414794,0.830466,2.250951,-0.435113,-0.424580,-0.897265,-1.055778,-0.132648,-0.480176,0.425161,-0.444743,1.122894,-0.364936,-0.148445,-0.486473,0.927250,0.109266,-0.281954,0.843450,0.037521,-0.133855,-0.145709,-0.723771,0.108867,-0.994056,0.130526,-0.991445
4748,-0.042946,-0.332135,-0.950320,-0.795425,-0.294097,-0.479494,-0.413183,0.010725,-0.398857,-0.312228,0.427879,1.177892,1.228912,0.560829,1.082235,0.759173,-0.328500,0.848600,2.250951,-0.464520,-0.424580,-0.897265,-1.055778,-0.052721,-0.480176,0.378101,-0.444743,1.187229,-0.364936,-0.129604,-0.486473,0.927250,-0.467522,-0.281954,0.853587,0.064512,-0.133855,-0.145709,-0.674306,0.087156,-0.996195,0.130526,-0.991445
4749,-0.085258,-0.302670,-0.580005,-0.901206,-0.294097,-0.450307,-0.442312,-0.034729,-0.427076,-0.284240,0.427879,1.177892,1.242267,-0.339752,0.863670,1.211967,-0.673675,0.880336,2.250951,-0.435113,-0.395567,-0.897265,-1.055778,-0.931921,-0.480176,0.378101,-0.415516,1.136680,-0.346967,-0.129604,-0.486473,0.927250,0.470138,-0.281954,0.878930,0.053716,-0.133855,-0.160915,-0.872164,0.065403,-0.997859,0.130526,-0.991445
4750,-0.148726,-0.302670,-0.580005,-0.689643,-0.294097,-0.450307,-0.384054,-0.057456,-0.370638,-0.284240,0.427879,1.177892,1.242267,1.652504,0.853360,-1.387470,-0.673675,0.912071,2.250951,-0.435113,-0.395567,-0.897265,-1.055778,-1.011848,-0.480176,0.307511,-0.415516,1.132085,-0.364936,-0.110764,-0.457661,0.927250,0

In [0]:

# ID로 시간 변수 생성
minute = pd.Series((test.id%144).astype(int))
hour= pd.Series((test.id%144/6).astype(int))

# 삼각함수를 이용한 시간변수 생성
min_in_day = 24*6
hour_in_day = 24

minute_sin = np.sin(np.pi*minute/min_in_day) 
minute_cos = np.cos(np.pi*minute/min_in_day)

hour_sin  = np.sin(np.pi*hour/hour_in_day)
hour_cos  = np.cos(np.pi*hour/hour_in_day)

In [0]:
X_test['minute_sin'] = minute_sin
X_test['minute_cos'] = minute_cos

X_test['hour_sin'] = hour_sin
X_test['hour_cos'] = hour_cos
X_test = X_test.drop(columns='id')

In [0]:
def mse1(y_pred, dataset):
    y_true = dataset.get_label()

    diff = abs(y_true - y_pred)
    less_then_one = np.array([0 if x < 1 else 1 for x in diff])

    y_pred = less_then_one * y_pred
    y_true = less_then_one *y_true
    
    score= mean_squared_error(y_true, y_pred)
    
    return 'score', score, False

In [0]:
X_train_norm = X_train_norm.loc[:,['X37', 'X38', 'X32','X31','X30','X20','X12','X07','X00','X40','X41','minute_sin','minute_cos','hour_sin','hour_cos']]
X_test = X_test.loc[:,['X37', 'X38', 'X32','X31','X30','X20','X12','X07','X00','X40','X41','minute_sin','minute_cos','hour_sin','hour_cos']]

In [0]:
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

model = xgb.XGBClassifier(feval=mse1)
params={ 'max_depth': [3], 'n_estimators':[4000],  'learning_rate': [0.007], \
        'boost_from_average': [False], 'subsample': [1], 'loss_function':['RMSE'], 'scale_pos_weight': [10], 'eta':[0.007],'slient':[1],
        'coal_sample_bytree':[0.75],'min_child_weight': [4], 'gamma': [0.4],'objective':['reg:linear'],'random_state':[777] }
cv=KFold(n_splits=6, random_state=1)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
gcv = GridSearchCV(model,param_grid=params, cv=cv, n_jobs=2,scoring='neg_mean_squared_error')

In [0]:
X_train_norm.values

array([[ 0.5849496 ,  0.25344775, -1.73101149, ...,  1.        ,
         0.        ,  1.        ],
       [ 0.59001823,  0.30203123, -1.80637182, ...,  0.99976203,
         0.        ,  1.        ],
       [ 0.59001823,  0.32362388, -1.91941231, ...,  0.99904822,
         0.        ,  1.        ],
       ...,
       [ 0.87893012,  0.05371566, -0.12960448, ..., -0.99785892,
         0.13052619, -0.99144486],
       [ 0.88906738,  0.10229914, -0.1107644 , ..., -0.99904822,
         0.13052619, -0.99144486],
       [ 0.92961642,  0.08610465, -0.07308423, ..., -0.99976203,
         0.13052619, -0.99144486]])

In [0]:



gcv.fit(X_train_norm,y)


print('final params', gcv.best_params_)
print('best score', gcv.best_score_)


In [0]:
model = gcv.best_estimator_
pred_y_mean = np.zeros([X_test.shape[0]])
X_train_18 = pd.concat([X_train_norm,y],axis=1)
X_train_18

In [0]:
X_train_18.loc[:,"X00":"hour_cos"]

In [0]:
cv = KFold(n_splits=6, random_state=24, shuffle=True)
i=0
pred_y_mean = np.zeros([X_test.shape[0]])
for tidx, vidx in cv.split(X_train_18):
  print(f'predict with Kfold {i}')
  i+=1
  #train_cv = X_train_18.iloc[tidx]
  #val_cv = X_train_18.iloc[vidx]
  train_x = X_train_18.iloc[:4320+4320,:-1]
  train_Y = X_train_18.iloc[:4320+4320, -1]
  val_x = X_train_18.iloc[4320+4320:, :-1]
  val_y = X_train_18.iloc[4320+4320:,-1]

  model.fit(train_x,train_Y,eval_set=[(val_x,val_y)],verbose=200)

  pred_y = model.predict(X_test)
  pred_y_mean = pred_y_mean + pred_y

In [0]:
pred_y_mean = pred_y_mean/6.0

In [0]:
sub['Y18'] = pred_y_mean
sub.to_csv("feval.csv",index=False)

In [0]:
pred_y_mean.max()